In [68]:
import torch as t
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from datasets import load_dataset 
from PIL import Image
import os, random
import pandas as pd
import transformers as tr
import json
import shutil
import copy

In [2]:
# DO NOT RUN CELL!

"""lines = []
with open("hateful_memes/train.jsonl") as f:
    lines = f.read().splitlines()

lines_dict = [json.loads(line) for line in lines]

df = pd.DataFrame(lines_dict)
print(df.loc[[0]])"""

      id            img  label  \
0  42953  img/42953.png      0   

                                               text  
0  its their character not their color that matters  


In [12]:
# DO NOT RUN CELL!
'''
imgs_referenced = df["img"]
imgs_referenced.astype(str)

def select_ref_imgs(orig_loc):
    orig_loc = str(orig_loc)
    file_name = orig_loc[4:]
   
    try:
        shutil.move(f".\hateful_memes\{orig_loc}", f".\hateful_memes\imgs_present/{file_name}")
        print(f"file {file_name} moved")
    except FileNotFoundError:
        print(f"file {file_name} not found")
        pass
    except Exception as e:
        print(f"error:{e}")

def change_slash(string):
    string = string.replace("/",  "\ " )
    string = string.replace(" ", "")
    return string
    '''

# imgs_referenced = imgs_referenced.apply(change_slash)
# imgs_referenced.apply(select_ref_imgs)

print("files moved")

files moved


In [13]:
# DO NOT RUN CELL!

# for file_name in os.listdir("./hateful_memes/img"):
    # file_path = os.path.join("./hateful_memes/img", file_name)
    # os.remove(file_path)
    # print(f"{file_path} deleted.")
    # pass
print ("files deleted")

files deleted


In [ ]:
# DO NOT RUN CELL!

# shutil.rmtree("hateful_memes/img_old")

In [7]:
# DO NOT RUN CELL!

# os.rename("hateful_memes/imgs_present", "hateful_memes/img")

In [14]:
# DO NOT RUN CELL!

"""def check_imgs_exist(file_path):
    file_path = str(file_path)
    file_name = file_path[4:]
    if os.path.isfile(f"hateful_memes/{file_path}"):
        # print(f"{file_name} present")
        return True
    else:
        return False
    
reference_df = df
reference_df["img_exists"] = reference_df["img"].apply(check_imgs_exist)
reference_df = reference_df[reference_df["img_exists"]]
reference_df = reference_df.drop(columns=["img_exists"])
reference_df = reference_df.reset_index()
"""

files present


In [4]:
# DO NOT RUN CELL!

# print(reference_df.head(), len(reference_df))

   index     id            img  label  \
0      0  42953  img/42953.png      0   
1      2  13894  img/13894.png      0   
2      4  82403  img/82403.png      0   
3      5  16952  img/16952.png      0   
4      6  76932  img/76932.png      0   

                                                text  
0   its their character not their color that matters  
1                           putting bows on your pet  
2  everybody loves chocolate chip cookies, even h...  
3           go sports! do the thing! win the points!  
4     fine you're right. now can we fucking drop it?   6744


In [ ]:
# DO NOT RUN CELL!

# print(reference_df.query("id == '14507'"))

     index     id            img  label  \
882   1112  14507  img/14507.png      1   

                                                  text  
882  when your history teacher starts talking about...  


In [15]:
# DO NOT RUN CELL!

# reference_df = reference_df.astype({"index":int, "id":int, "img":str, "label":float, "text":str})
# data = reference_df

# data.to_csv("data.csv")

In [2]:
data = pd.read_csv("data.csv")
print (data.head())

   Unnamed: 0  index     id            img  label  \
0           0      0  42953  img/42953.png    0.0   
1           1      2  13894  img/13894.png    0.0   
2           2      4  82403  img/82403.png    0.0   
3           3      5  16952  img/16952.png    0.0   
4           4      6  76932  img/76932.png    0.0   

                                                text  
0   its their character not their color that matters  
1                           putting bows on your pet  
2  everybody loves chocolate chip cookies, even h...  
3           go sports! do the thing! win the points!  
4     fine you're right. now can we fucking drop it?  


In [3]:
class DatasetTxt (Dataset):
    def __init__(self, df, tokenizer):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer

        self.text= df["text"].values
        self.label = df["label"].values

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        text = self.text[index]
        label = self.label[index]

        encoding = self.tokenizer.encode_plus(text, add_special_tokens=True, return_token_type_ids=False,
            padding='longest', return_attention_mask=True, return_tensors='pt', truncation=True)

        return {"input_ids": encoding["input_ids"].flatten(), "attention_mask": encoding["attention_mask"].flatten(),
                "labels": t.tensor(label, dtype=t.long)}
        
class DatasetImg (Dataset):
    def __init__(self, df, img_dir):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ])

        self.img= df["img"].values
        self.label = df["label"].astype(int).values

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        img_pth = os.path.join(self.img_dir, self.img[index])
        img = Image.open(img_pth).convert("RGB")
        img_tensor = self.transform(img).unsqueeze(0)
        img_tensor = img_tensor.squeeze(1)
        label = self.label[index]

        return img_tensor, label



In [4]:
# splitting data 50%
train_data, rest = train_test_split(data, test_size= 0.5)

In [5]:
tokenizer = tr.BertTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english", clean_up_tokenization_spaces=True)

In [6]:
text_ds = DatasetTxt(train_data, tokenizer=tokenizer)

In [7]:
imag_ds = DatasetImg(train_data, img_dir="hateful_memes/")

In [8]:
d_collator = tr.DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
train_loader_txt = DataLoader(text_ds, batch_size=16, shuffle=True, collate_fn=d_collator)
train_loader_img = DataLoader(imag_ds, batch_size=16, shuffle=True)

In [81]:
class MonoLateFusModel(nn.Module):
    def __init__(self, text_model, img_model, text_dims, img_dims):
        super().__init__()
        self.text_model = text_model
        self.img_model = img_model


        # architecture
        #ToxicBERT/HateBERT input
        self.textfc = nn.Linear(text_dims, 128)
        #ResNet input
        self.imgfc = nn.Linear(img_dims, 128)
        self.mlp = nn.Sequential(nn.Linear(128, 16), nn.ReLU(), nn.Linear(16, 2))

    def forward(self, tokenized_text, image):
        #to be done
        cloned_txt_model =copy.deepcopy(self.text_model)
        cloned_img_model = copy.deepcopy(self.img_model)
        
        toktext = tokenized_text["input_ids"]
        mask = tokenized_text["attention_mask"]
        labels = tokenized_text["labels"]

        # txt_proc = self.textfc(self.text_model(tokenized_text))
        # img_proc = self.imgfc(self.img_model(image))

        logits_for_txt = cloned_txt_model(input_ids=toktext, attention_mask=mask, labels=labels).logits
        img_t = cloned_img_model(image)

        txt_proc = self.textfc(logits_for_txt)
        img_proc = self.imgfc(img_t)

        txt_fft = t.fft.fft(txt_proc)
        img_fft = t.fft.fft(img_proc)

        fusion = t.real(txt_fft * img_fft)
        logits = self.mlp(fusion)

        # print("image: ", img_t)
        # print("text output: ", logits_for_txt)
        #print("Text proc output:", txt_proc)
        #print("Image proc output:", img_proc)
        #print("Txt FFT:", txt_fft)
        #print("Img FFT:", img_fft)
        #print("Fusion output:", fusion)
        # combined_outputs = t.cat([txt_proc, img_proc], 1)
        # outputs = self.fusion(combined_outputs)

        return logits
    
    def fine_tune_subpart(self, dataloader, input_type):
        model = self.text_model if input_type == "txt" else self.img_model
        model.train()


        tot_loss = 0
        correct_preds = 0
        #tot_val = 0

        if input_type == "txt":
            for name, param in model.named_parameters():
                if "classifier" not in name:
                    param.requires_grad = False
            optimizer_txt = optim.AdamW(model.parameters(), lr= 5e-5)
            scheduler = optim.lr_scheduler.StepLR(optimizer_txt, step_size=1, gamma=0.9)
            
        elif input_type == "img":
            for name, param in model.named_parameters():
                if "fc" not in name:
                    param.requires_grad = False
            criterion = nn.CrossEntropyLoss()
            optimizer_img = optim.Adam(model.fc.parameters(), lr=0.001)

        for e in range(3):
            if input_type == "txt":
                for batch in dataloader:
                    toktext = batch["input_ids"]
                    mask = batch["attention_mask"]
                    labels = batch["labels"]


                    outputs = model(input_ids=toktext, attention_mask=mask, labels=labels)

                    # loss= nn.CrossEntropyLoss()

                    loss = outputs.loss
                    logits = outputs.logits

                    optimizer_txt.zero_grad()
                    loss.backward()
                    optimizer_txt.step()
                    scheduler.step()
                    tot_loss += loss.item()

                    pred = t.argmax(logits, -1)
                print(f"LOSS: {loss.item()}, PRED: {pred}")
                    

            elif input_type == "img":
                for images, labels in dataloader:

                    images = images.squeeze(1)

                    optimizer_img.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs.float(), labels)
                    loss.backward()
                    optimizer_img.step()
                    tot_loss += loss.item()
                    
                    #tot_val += labels.size(0)
                    
                    _, pred = t.max(outputs.data, 1)
                print(f"LOSS: {loss.item()}, PRED: {pred}")

            
            else:
                print ("input_type must be either 'txt' or 'img'")
                
            correct_preds = t.sum(pred == labels)
            print(f"batch {e+1}/3", 
                  f"loss: {tot_loss/(len(dataloader)*(e+1)):.4f}, accuracy: {100*(correct_preds/(len(dataloader))):.4f}")
        return "loss:", tot_loss/(len(dataloader)*(e+1)), "accuracy:", 100*(correct_preds/(len(dataloader)))

In [70]:
model_txt = tr.AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english", num_labels = 2)
model_img = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

In [82]:
hmc_lf_model = MonoLateFusModel(text_model=model_txt, img_model=model_img, text_dims=2, img_dims=1000)

In [84]:
hmc_lf_model.fine_tune_subpart(train_loader_txt, "txt")

LOSS: 1.0692874193191528, PRED: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
batch 1/3 loss: 0.7106, accuracy: 3.3175
LOSS: 0.6430527567863464, PRED: tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
batch 2/3 loss: 0.7130, accuracy: 4.2654
LOSS: 0.8184334635734558, PRED: tensor([0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])
batch 3/3 loss: 0.7149, accuracy: 3.3175


('loss:', 0.7148623623271689, 'accuracy:', tensor(3.3175))

In [73]:
hmc_lf_model.fine_tune_subpart(train_loader_img, "img")

LOSS: 0.8351051211357117, PRED: tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0])
batch 1/3 loss: 1.2329, accuracy: 3.7915
LOSS: 0.7963578104972839, PRED: tensor([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])
batch 2/3 loss: 0.9827, accuracy: 3.3175
LOSS: 0.5978100895881653, PRED: tensor([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])
batch 3/3 loss: 0.8787, accuracy: 4.2654


('loss:', 0.8786777587486858, 'accuracy:', tensor(4.2654))

In [48]:
class dataset_hmc (Dataset):
    def __init__(self, df, tokenizer, img_path):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer
        self.img_dir = img_path
        self.transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ])

        self.text= df["text"].values
        self.label = df["label"].values
        self.img = df["img"].values

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        
        text = self.text[index]
        label = self.label[index]

        encoding = self.tokenizer.encode_plus(text, add_special_tokens=True, return_token_type_ids=False,
            padding='longest', return_attention_mask=True, return_tensors='pt', truncation=True)
        
        img_pth = os.path.join(self.img_dir, self.img[index])
        img = Image.open(img_pth).convert("RGB")
        img_tensor = self.transform(img).unsqueeze(0)
        img_tensor = img_tensor.squeeze(1)
        label = self.label[index]

        return {"input_ids": encoding["input_ids"].flatten(), 
                "attention_mask": encoding["attention_mask"].flatten(),
                "img_tensor": img_tensor,
                "labels": t.tensor(label, dtype=t.long),
            }

In [49]:
train_hmc, test = train_test_split(rest, test_size=0.3)

In [50]:
ds_hmc = dataset_hmc(df=train_hmc, tokenizer=tokenizer, img_path="hateful_memes/")

In [51]:
dataloader_hmc = DataLoader(ds_hmc, batch_size=16, shuffle=True, collate_fn=d_collator)

In [83]:
hmc_lf_model.img_model.train()
for name, param in hmc_lf_model.img_model.named_parameters():
    if "fc" not in name:
        param.requires_grad = False
    if not param.requires_grad and param.grad is not None:
        print(f"Warning: {name} has a gradient despite being frozen.")
    #print(f"{name}: requires_grad={param.requires_grad}")
hmc_lf_model.text_model.train()
for name, param in hmc_lf_model.text_model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False
    if not param.requires_grad and param.grad is not None:
        print(f"Warning: {name} has a gradient despite being frozen.")
    # print(f"{name}: requires_grad={param.requires_grad}")

for name, param in hmc_lf_model.named_parameters():
    if not param.requires_grad and param.grad is not None:
        print(f"Warning: {name} has a gradient despite being frozen.")
    #print(f"{name}: requires_grad={param.requires_grad}")

hmc_lf_model.train()


total_loss = 0
correct_preds = 0

loss_function = nn.CrossEntropyLoss()
optimizer_general = t.optim.AdamW(filter(lambda p: p.requires_grad, hmc_lf_model.parameters()), 
    lr=1e-5
)
# optimizer_img = optim.Adam(hmc_lf_model.img_model.fc.parameters(), lr=0.001)
# optimizer_txt = optim.AdamW(hmc_lf_model.text_model.parameters(), lr= 5e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer_txt, step_size=1, gamma=0.9)

for e in range(3):
    for batch in dataloader_hmc:
        
        text_data = {
        "input_ids": batch["input_ids"],
        "attention_mask": batch["attention_mask"],
        "labels": batch["labels"]
        }
        img = batch["img_tensor"]
        img_squeezed = img.squeeze(1)
        label = batch["labels"]
        # print(text_data)

        outputs = hmc_lf_model(text_data,img_squeezed)
        #print (outputs)
        loss = loss_function(outputs, label)

        optimizer_general.zero_grad()
        loss.backward()
        optimizer_general.step()
        #print(f"loss: {loss.item()}")
        total_loss += loss.item()
        _, pred = t.max(outputs.data, 1)

        correct_preds += t.sum(pred == label)

    print(f"batch {e+1}/3", 
    f"loss: {total_loss/(len(dataloader_hmc)*(e+1)):.4f}, accuracy: {correct_preds.float()/len(dataloader_hmc):.4f}")

batch 1/3 loss: 7.3669, accuracy: 9.5203
batch 2/3 loss: 6.9181, accuracy: 18.7297
batch 3/3 loss: 6.3881, accuracy: 28.1824
